In [16]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BartForConditionalGeneration, AutoTokenizer, BartTokenizer
import torch.optim
import accelerate
import os
import json
from tqdm import tqdm
from time import time
from random import sample
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import bleu_score
import nltk
from rouge import Rouge
from pycocoevalcap.cider.cider import Cider

In [2]:
data_root_path = os.path.join(os.getcwd(), 'v3/v2/en-hi')
model_path = os.path.join(os.path.join(os.getcwd(), 'Models'), 'Machine_Translation')

In [3]:
class CustomDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        #self.data = self.load_data(data_path, sample_size)
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        #self.sample_size = sample_size
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['English']
        target_text = item['Hindi']
        
        inputs = self.tokenizer.encode_plus(input_text, max_length=self.max_length,
                                            padding='max_length', truncation=True, return_tensors='pt'
                                           )
        
        targets = self.tokenizer.encode(target_text, max_length=self.max_length,
                                        padding='max_length', truncation=True, return_tensors='pt'
                                        )
        
        return {'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': targets.squeeze()
               }
    """def load_data(self, data_path, sample_size):
        '''with open(data_path, 'r',encoding="utf8") as file:
            data = json.load(file)'''
        data = []
        with open(os.path.join(data_root_path, 'train.en'), 'r', encoding="utf8") as en_file:
            en_lines = en_file.readlines()
        with open(os.path.join(data_root_path, 'train.hi'), 'r', encoding="utf8") as hn_file:
            hn_lines = hn_file.readlines()
            
        for en_line, hn_line in zip(en_lines, hn_lines):
            data.append(dict(English=en_line.replace('\n', ''), Hindi=hn_line.replace('\n', '')))
            
        return sample(data, sample_size)"""

In [4]:
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)

In [5]:
def load_data(data_path, sample_size):
    '''with open(data_path, 'r',encoding="utf8") as file:
        data = json.load(file)'''
    data = []
    with open(os.path.join(data_root_path, 'train.en'), 'r', encoding="utf8") as en_file:
        en_lines = en_file.readlines()
    with open(os.path.join(data_root_path, 'train.hi'), 'r', encoding="utf8") as hn_file:
        hn_lines = hn_file.readlines()

    for en_line, hn_line in zip(en_lines, hn_lines):
        data.append(dict(English=en_line.replace('\n', ''), Hindi=hn_line.replace('\n', '')))

    return sample(data, sample_size)

In [6]:
data = load_data(data_root_path, 5600)

In [7]:
train_data, test_data = train_test_split(data, test_size=0.1785, shuffle=False)
train_data, val_data = train_test_split(train_data, test_size=0.1304, shuffle=False)

In [8]:
train_dataset = CustomDataset(tokenizer, train_data)
dev_dataset = CustomDataset(tokenizer, val_data)
test_dataset = CustomDataset(tokenizer, test_data)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
val_loader = DataLoader(dev_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [10]:
def Fine_Tune(train_loader, val_loader, num_epochs, num_layers_to_finetune):
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    
    for param in model.parameters():
        param.requires_grad = False
    
    for i in range(1, num_layers_to_finetune + 1):
        for param in model.model.encoder.layers[-i].parameters():
            param.requires_grad = True
        for param in model.model.decoder.layers[-i].parameters():
            param.requires_grad = True
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print(device)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=7e-5, weight_decay=3e-6)
    for epoch in tqdm(range(num_epochs)):
        model.train()
        print("Training phase")
        #for idx, batch in enumerate(train_loader):
        for batch in tqdm(train_loader):
            #time0 = time()
            #print(f"batch: {idx+1} starts")
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            #time1 = time()
            #print(f"batch: {idx+1} ends\ntime taken: {time1-time0} seconds")
            #print(f"time taken: {time1-time0} seconds")
        
        model.eval()
        print("Validation phase")
        with torch.no_grad():
            total_val_loss = 0.0
            for val_batch in tqdm(val_loader):
                input_ids = val_batch['input_ids'].to(device)
                attention_mask = val_batch['attention_mask'].to(device)
                labels = val_batch['labels'].to(device)
                val_output = model(input_ids, attention_mask=attention_mask, labels=labels)
                total_val_loss += val_output.loss.item()
            average_val_loss = total_val_loss / len(val_loader)
            
        print(f'Epoch: {epoch+1}/{num_epochs}, Validation Loss: {average_val_loss}')
    os.makedirs(model_path, exist_ok=True)
    model.save_pretrained(model_path, from_pt=True)
    torch.save(model.state_dict(), os.path.join(model_path, 'pytorch_model.bin'))

In [20]:
def Generate_translation_show(val_loader):
    model = BartForConditionalGeneration.from_pretrained(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    with torch.no_grad():
        for val_batch in val_loader:
            input_ids = val_batch['input_ids'].to(device)
            attention_mask = val_batch['attention_mask'].to(device)
            labels = val_batch['labels'].to(device)
            generated_ids = model.generate(input_ids, attention_mask=attention_mask,
                                           max_length=60, num_beams=2, repetition_penalty=2.0,
                                           length_penalty=2.0, early_stopping=True
                                          )
            generated_title = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            actual_title = tokenizer.decode(labels[0], skip_special_tokens=True)
            
            print(f'Input Text: {input_text}\nGenerated Translation: {generated_title}\nActual Translation: {actual_title}')
            print('\n'+'='*50+'\n')

In [24]:
def Generate_translation(val_loader):
    model = BartForConditionalGeneration.from_pretrained(model_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    generated_translations = []
    actual_translations = []
    with torch.no_grad():
        for val_batch in val_loader:
            input_ids = val_batch['input_ids'].to(device)
            attention_mask = val_batch['attention_mask'].to(device)
            labels = val_batch['labels'].to(device)
            generated_ids = model.generate(input_ids, attention_mask=attention_mask,
                                           max_length=60, num_beams=2, repetition_penalty=2.0,
                                           length_penalty=2.0, early_stopping=True
                                          )
            generated_translation = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
            actual_translation = tokenizer.decode(labels[0], skip_special_tokens=True)
            generated_translations.append(generated_translation)
            actual_translations.append(actual_translation)
    return generated_translations, actual_translations

In [17]:
def calculate_bleu_scores(actual, generated):
    smoothie = SmoothingFunction().method4
    actual_tokenized = [[nltk.word_tokenize(act) for act in group] for group in actual]
    generated_tokenized = [nltk.word_tokenize(gen) for gen in generated]
    score = corpus_bleu(actual_tokenized, generated_tokenized, smoothing_function=smoothie)
    return score

In [18]:
def calculate_rouge_scores(actual, generated):
    rouge = Rouge()
    scores = rouge.get_scores(generated, actual)
    return scores

In [19]:
def calculate_cider_scores(actual, generated):
    act_dict = {idx: [line] for idx, line in enumerate(actual)}
    gen_dict = {idx: [line] for idx, line in enumerate(generated)}
    cider = Cider()
    (score, scores) = cider.compute_score(act_dict, gen_dict)
    return score

In [12]:
Fine_Tune(train_loader, val_loader, 5, 2)

  0%|                                                                                                                                                                                                                                                   | 0/5 [00:00<?, ?it/s]

Training phase



  0%|                                                                                                                                                                                                                                                 | 0/160 [00:00<?, ?it/s]C:\Users\udaya\miniconda3\envs\torch\Lib\site-packages\transformers\models\bart\modeling_bart.py:590: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(

 17%|███████████████████████████████████████▏                                                                                                                                                                                                | 27/160 [03:24<16:56,  7.64s/it]


 35%|█████████████████████████████████████████████████████████████████████████████████▏                                                                                                                                                      | 56/160 [07:06<13:15,  7.65s/it]


 53%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                            | 85/160 [10:47<09:32,  7.63s/it]


...

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                       | 144/160 [15:00<01:40,  6.29s/it]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [16:40<00:00,  6.26s/it]


Validation phase



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [1:31:22<00:00, 1096.52s/it]
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Epoch: 5/5, Validation Loss: 3.4473043779532113


In [25]:
generated_translations, actual_translations = Generate_translation(test_loader)

Input Text: <s>If the two mad men had opened their hearts to each other, then the wise people would have been puzzled.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Input Text: <s>The premeditated plan to alter the condition of the ball during Saturdays third day of the third Test against South Africa at Newlands dominated news headlines and captain Steve Smith and his teammates were widely condemned.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

Input Text: <s>You will get some good news.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Input Text: <s>He said officials were checking if there were any casualties or damage.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

Input Text: <s>Taking suo motu cognizance of Hajelas interview to a national daily, Justice Gogoi said: Where is the necessity, scope and authority in you to make such a statement?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

...

Input Text: <s>India has its first individual Olympics gold medal.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

Input Text: <s>The deceased was a labourer.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Input Text: <s>"""Bring them back to me!"" - - [he said] and began to stroke their legs and their necks."</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

Input Text: <s>Two of Diamonds</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

Input Text: <s>Candidates will be selected on the basis of skill test.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [28]:
bleu_score = calculate_bleu_scores(actual_translations, generated_translations)

In [29]:
rouge_score = calculate_rouge_scores(actual_translations, generated_translations)

In [30]:
cider_score = calculate_cider_scores(actual_translations, generated_translations)

In [31]:
bleu_score, rouge_score, cider_score

(0.0030682112588578466,
 [{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
  {'rouge-1': {'r': 0.04, 'p': 0.125, 'f': 0.060606056932966244},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.04, 'p': 0.125, 'f': 0.060606056932966244}},
  {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
  {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}},
  {'rouge-1': {'r': 0.03571428571428571, 'p': 0.125, 'f': 0.05555555209876565},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.03571428571428571,
    'p': 0.125,
    'f': 0.05555555209876565}},
  {'rouge-1': {'r': 0.08571428571428572,
    'p': 0.3333333333333333,
    'f': 0.13636363310950422},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r'